In [ ]:
''''
Raport z uczenia maszynowego.

1. Wczytanie bibliotek wykorzystywanych w projekcie
2. Wczytanie danych, usunięcie wartości niedozwolonych, usunięcie duplikatów res_name i pdb_code oraz
pozostawienie klas o liczności >=5 
3. Wstępne przetwarzanie danych: pozostawienie kolumn numerycznych, usunięcie kolumn zawierających same NaN oraz
kolumn wypełnionych jedną wartością.
4. Wczytywanie danych testowych i poddanie i przetwarzaniu analogicznemu do punktu 3. Usunięcie ze zbioru uczącego niedostępnych
w zbiorze testowym kolumn. Kolumna 'local_res_atom_O_count' nie została usunięta, ponieważ wtedy klasyfikator przestawał 
działać. Nie potrafiłam naprawić tego problemu, więc dodałam do zbioru uczącego kolumnę wypełnioną zerami, zapewne pogorszy to 
nieco wynik klasyfikacji. Wypełnienie wartości NaN wartościami średnimi kolumn.
5. Tworzenie klasyfikatora z etykiet res_name. Aby skorzystać z funkcji recall_score() należało odpowiednio przetransformować 
etykiety. Następnie korzystałam z RandomForestClassifier() oraz funkcji RandomizedSearchCV() w celu zoptymalizowania parametrów.
Dla zbioru danych, który zawierał kolumny brakujące w zbiorze testowym, recall utrzymywał się na poziomie powyżej 0.82. Po 
usunięciu tych kolumn miara spadła do 0.47.
6. Tworzenie klasyfikatora z etykiet res_name_grouped. Dla zbioru danych, który zawierał kolumny brakujące w zbiorze testowym, 
recall utrzymywał się na poziomie powyżej 0.89. Pousunięciu tych kolumn miara spadła do 0.51.
7. Predykcja wartości dla zbioru testowego i zapis do pliku.
8. Serializacja za pomocą biblioteki joblib. Parametry zostały podane ręcznie dla najlepszych uzyskanych wyników podczas testów.
''''

In [1]:
#Projekt powinien być napisany w języku Python z wykorzystaniem bibliotek Pandas i scikit-learn
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint
from sklearn.externals import joblib

In [2]:
#wczytywanie danych z pliku
df = pd.read_csv("all_summary.csv", sep=";", na_values=["n/a"])
'''
Ze zbioru danych powinny zostać usunięte wiersze posiadające wartość zmiennej 
res_name równą: “DA”,“DC”,“DT”, “DU”, “DG”, “DI”,“UNK”, “UNX”, “UNL”, “PR”, “PD”, “Y1”, “EU”, “N”, “15P”, “UQ”, “PX4” lub “NAN”;
'''
forbidden_values = ["DA","DC","DT", "DU", "DG", "DI","UNK", "UNX", "UNL", "PR", "PD", "Y1", "EU", "N", "15P", "UQ", "PX4", "NAN"]
df = df[~df.res_name.isin(forbidden_values)]

#W zbiorze danych uczących powinny zostać tylko unikatowe pary (pdb_code, res_name)
df = df.drop_duplicates(['res_name', 'pdb_code'])

#Klasyfikator uczony na oryginalnym zbiorze etykiet (res_name) powinien korzystać ze wszystkich klas,
#których liczność wynosi co najmniej 5 przykładów; klasy o mniejszej liczności nie mają być brane pod uwagę
value_counts = df['res_name'].value_counts()
to_remove = value_counts[value_counts<5].index
liczba = value_counts[value_counts<5]
df = df[~df.res_name.isin(to_remove)]

C:\Users\Mae\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\io\parsers.py:1159: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
#wstępne przetwarzanie

X = df.drop('res_name', axis=1)
#zostawia tylko kolumny numeryczne
X = X._get_numeric_data()
#pozbywa się kolumn wypełnonych NaN
X = X.drop(["local_BAa", "local_NPa", "local_Ra", "local_RGa", "local_SRGa", "local_CCPa", "local_ZOa", "local_ZDa", "local_ZD_minus_a", "local_CCSa", "local_ZD_plus_a", "weight_col"], axis=1)
#pozbywa się kolumn wypełnionych w całości tymi samymi wartościami
X = X.drop(["local_min", "grid_space", "solvent_radius", "solvent_opening_radius", "resolution_max_limit", "part_step_FoFc_std_min", "part_step_FoFc_std_max", "part_step_FoFc_std_step" ], axis=1)
#print(X.columns.get_loc("part_00_blob_electron_sum"))
#print(X.columns.get_loc("part_09_density_sqrt_E3"))
#usuwanie kolumn zaczynających się od part_0X, z uwagi na dużo wartości NaN i wzajemną korelację
X = X.drop(X.columns[15:705], axis=1)

In [4]:
#wczytywanie danych do predykcji wartości
td = pd.read_csv("test_data.txt", sep=",", na_values=["n/a"])

#print(test_data.columns.get_loc("part_00_blob_electron_sum"))
#print(test_data.columns.get_loc("part_10_density_sqrt_E3"))

#oczyszczanie kolumn z partXX
X_td = td.drop(td.columns[1:760], axis=1)

#zostawia tylko kolumny numeryczne
X_td = X_td._get_numeric_data()
#pozbywa się kolumn wypełnionych w całości tymi samymi wartościami i id
X_td = X_td.drop(["local_min", "Unnamed: 0"], axis=1)

#tworzenie list niedostępnych kolumn
unavailable_columns=[]
for col in X.columns.values:
    if col not in X_td.columns.values:
        unavailable_columns.append(col)
        
    
#usuwanie ze zbioru uczącego niedostępych kolumn
X = X.drop(['local_res_atom_count','local_res_atom_non_h_count', 'local_res_atom_non_h_occupancy_sum', 'local_res_atom_non_h_electron_sum', 'local_res_atom_non_h_electron_occupancy_sum', 'local_res_atom_C_count', 'local_res_atom_N_count', 'local_res_atom_S_count', 'dict_atom_non_h_count', 'dict_atom_non_h_electron_sum', 'dict_atom_C_count', 'dict_atom_N_count', 'dict_atom_O_count', 'dict_atom_S_count'], axis=1)

'''
Kolumna 'local_res_atom_O_count' nie została usunięta, ponieważ wtedy przestawał działać random_forest, 
nie potraiłam sobie poradzić z tym błędem. Dlatego zdecydowałam się na dodawnie kolumny wypełnionej zerami
do testowego data frame'a. Może to negatywnie wpłynąć na wyniki klasyfikacji.

'''
column = pd.DataFrame()
column['local_res_atom_O_count'] = pd.Series(0, index=X_td.index)
X_td = pd.concat([column, X_td], axis=1)

#wypełnianie wartości NaN wartościami średnimi (maksymalnie stanowią ok. 1% wartości kolumn wynosi NaN)
X = X.fillna(X.mean())
y = df["res_name"]

In [23]:
#klasyfikator na etykietach res_name

#transformacja etykiet, konieczna do skorzystania z funkcji recall_score()
le = preprocessing.LabelEncoder()
le.fit(y.values)

#podział na zbiór testowy i treninogwy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# budowanie klasyfikatora random forest
clf1 = RandomForestClassifier(n_estimators=20)

# specifikowanie parametrów do randomized search
param_dist1 = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# odpalenie randomized search
n_iter_search1 = 20
random_search1 = RandomizedSearchCV(clf1, param_distributions=param_dist1, n_iter=n_iter_search1)

#uzyskanie najlepszych parametrów
forest = random_search1.fit(X_train, y_train)
#print(random_search1.grid_scores_)

#predykcja dla zbioru testowego
y_pred=forest.predict(X_test)
y_true=y_test.values
y_true_norm=le.transform(y_true)
y_pred_norm=le.transform(y_pred)

#estymacja miary recall
print(recall_score(y_true_norm, y_pred_norm, average='weighted'))

[mean: 0.16864, std: 0.01177, params: {'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 10, 'criterion': 'gini', 'max_features': 2, 'max_depth': 3}, mean: 0.42282, std: 0.01711, params: {'bootstrap': True, 'min_samples_leaf': 2, 'min_samples_split': 4, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}, mean: 0.42970, std: 0.02214, params: {'bootstrap': False, 'min_samples_leaf': 8, 'min_samples_split': 7, 'criterion': 'gini', 'max_features': 9, 'max_depth': None}, mean: 0.33688, std: 0.01361, params: {'bootstrap': False, 'min_samples_leaf': 8, 'min_samples_split': 8, 'criterion': 'gini', 'max_features': 3, 'max_depth': None}, mean: 0.25964, std: 0.00972, params: {'bootstrap': True, 'min_samples_leaf': 5, 'min_samples_split': 5, 'criterion': 'gini', 'max_features': 8, 'max_depth': 3}, mean: 0.24809, std: 0.01513, params: {'bootstrap': False, 'min_samples_leaf': 5, 'min_samples_split': 4, 'criterion': 'gini', 'max_features': 5, 'max_depth': 3}, mean: 0.30624, std

In [25]:
#klasyfikator na etykietach zgrupowanych

#wczytanie etykiet zgrupowanych
df_grouped = pd.read_csv("grouped_res_name.txt", sep=",", na_values=["n/a"])
y_grouped = df_grouped["res_name_group"]

#normalizacja etykiet zgrupowanych
le_grouped = preprocessing.LabelEncoder()
le_grouped.fit(y_grouped.values)

#podział na zbiór testowy i treninogwy dla etykiet zgrupowanych
X_train_grouped, X_test_grouped, y_train_grouped, y_test_grouped = train_test_split(X, y_grouped, test_size=0.3, random_state=0)

# budowanie klasyfikatora random forest
clf_grouped = RandomForestClassifier(n_estimators=20)

# specifikowanie parametrów do randomized search
param_dist_grouped = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# odpalenie randomized search
n_iter_search_grouped = 20
random_search_grouped = RandomizedSearchCV(clf_grouped, param_distributions=param_dist_grouped, n_iter=n_iter_search_grouped)

#uzyskanie najlepszych parametrów dla etykiet zgrupowanych
forest_grouped = random_search_grouped.fit(X_train_grouped, y_train_grouped)
print(random_search_grouped.grid_scores_)

#predykcja dla zbioru testowego etykiet zgrupowanych
y_pred_grouped=forest_grouped.predict(X_test_grouped)
y_true_grouped = y_test_grouped.values
y_true_grouped_norm=le_grouped.transform(y_true_grouped)
y_pred_grouped_norm=le_grouped.transform(y_pred_grouped)

#estymacja miary recall
print(recall_score(y_true_grouped_norm, y_pred_grouped_norm, average='weighted'))

[mean: 0.41568, std: 0.02349, params: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 10, 'criterion': 'entropy', 'max_features': 5, 'max_depth': None}, mean: 0.28690, std: 0.01117, params: {'bootstrap': True, 'min_samples_leaf': 2, 'min_samples_split': 3, 'criterion': 'entropy', 'max_features': 7, 'max_depth': 3}, mean: 0.25094, std: 0.00689, params: {'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 4, 'max_depth': 3}, mean: 0.49137, std: 0.02595, params: {'bootstrap': False, 'min_samples_leaf': 7, 'min_samples_split': 6, 'criterion': 'entropy', 'max_features': 10, 'max_depth': None}, mean: 0.29170, std: 0.01879, params: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 9, 'criterion': 'gini', 'max_features': 7, 'max_depth': 3}, mean: 0.48332, std: 0.03682, params: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 10, 'criterion': 'entropy', 'max_features': 9, 'max_depth': None}, mean

In [27]:
#predykcja wyników dla test_data.txt

X_td = X_td.fillna(X_td.mean())

y_test_pred=forest.predict(X_td)

file_ = open('predykcja1.txt','w')
j=0
file_.write('"","res_name"\n')
for i in y_test_pred:
    file_.write('"{}","{}"\n'.format(j,i))
    j += 1
file_.close()


y_test_pred_grouped=forest_grouped.predict(X_td)

file_ = open('predykcja2.txt','w')
j=0
file_.write('"","res_name_group"\n')
for i in y_test_pred_grouped:
    file_.write('"{}","{}"\n'.format(j,i))
    j += 1
file_.close()

In [26]:
'''
Randomized search za każdym razem zwraca inne wyniki, zatem utworzyłam ręcznie klasyfikatory do serializacji z parametrów,
które dawały najlepsze wyniki.
Dla res_name recall wynosi: 0.470926711084
Dla res_name_grouped recall wynosi: 0.516656571775
'''

clf1_bestparams = RandomForestClassifier(n_estimators=20, bootstrap=True, min_samples_leaf = 2, min_samples_split=4, criterion='gini', max_features=10, max_depth= None)
clf_grouped_bestparams = RandomForestClassifier(n_estimators=20, bootstrap=False, min_samples_leaf = 7, min_samples_split=6, criterion='entropy', max_features=10, max_depth= None)
clf1_bestparams.fit(X_train, y_train)
clf_grouped_bestparams.fit(X_train_grouped, y_train_grouped)
    
joblib.dump(clf1_bestparams, 'clf1.pkl')
joblib.dump(clf_grouped_bestparams, 'clf2.pkl')


['clf2.pkl',
 'clf2.pkl_01.npy',
 'clf2.pkl_02.npy',
 'clf2.pkl_03.npy',
 'clf2.pkl_04.npy',
 'clf2.pkl_05.npy',
 'clf2.pkl_06.npy',
 'clf2.pkl_07.npy',
 'clf2.pkl_08.npy',
 'clf2.pkl_09.npy',
 'clf2.pkl_10.npy',
 'clf2.pkl_11.npy',
 'clf2.pkl_12.npy',
 'clf2.pkl_13.npy',
 'clf2.pkl_14.npy',
 'clf2.pkl_15.npy',
 'clf2.pkl_16.npy',
 'clf2.pkl_17.npy',
 'clf2.pkl_18.npy',
 'clf2.pkl_19.npy',
 'clf2.pkl_20.npy',
 'clf2.pkl_21.npy',
 'clf2.pkl_22.npy',
 'clf2.pkl_23.npy',
 'clf2.pkl_24.npy',
 'clf2.pkl_25.npy',
 'clf2.pkl_26.npy',
 'clf2.pkl_27.npy',
 'clf2.pkl_28.npy',
 'clf2.pkl_29.npy',
 'clf2.pkl_30.npy',
 'clf2.pkl_31.npy',
 'clf2.pkl_32.npy',
 'clf2.pkl_33.npy',
 'clf2.pkl_34.npy',
 'clf2.pkl_35.npy',
 'clf2.pkl_36.npy',
 'clf2.pkl_37.npy',
 'clf2.pkl_38.npy',
 'clf2.pkl_39.npy',
 'clf2.pkl_40.npy',
 'clf2.pkl_41.npy',
 'clf2.pkl_42.npy',
 'clf2.pkl_43.npy',
 'clf2.pkl_44.npy',
 'clf2.pkl_45.npy',
 'clf2.pkl_46.npy',
 'clf2.pkl_47.npy',
 'clf2.pkl_48.npy',
 'clf2.pkl_49.npy',
 'clf2.